In [57]:
import requests
import urllib
from bs4 import BeautifulSoup
from collections import Counter
import pandas as pd


In [58]:
lst_url =[
'https://www.gurufocus.com/guru/consensus-picks'
    
    ]



In [59]:
lst_Merge_Guru = []


In [60]:
# GET TICKERS FROM ALL CAPS
for link in range(len(lst_url)):
	headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:101.0) Gecko/20100101 Firefox/101.0', "cookie": "CONSENT=YES+cb.20230531-04-p0.en+FX+908"}

	response = requests.get(lst_url[link ], headers=headers )

	if response.status_code != 200:
		print("Error fetching page")
		exit()
	else:
		content = response.content
		
	soup = BeautifulSoup(response.content, 'html.parser')
	# All links in the page

	nb_links = len(soup.find_all('a'))
	print(f"There are {nb_links} links in this page")

	# GET ALL Text from the LINKS
	items = [item.text.strip() for item in soup.select('a') ]
	# for i in items:
	# 	print(i)

	#   REMOVE NON LINK TEXT
	test_list = [item.text.strip() for item in soup.select('a') ]
	# print(test_list)


	#  FILTER FOR CAPS ONLY
	for word in items:
		upperChars = True
		for letters in word:
	
			# checking for uppercase
			if not letters.isupper():
				upperChars = False
				break
		if upperChars:
			lst_Merge_Guru.append(word)
	

	# remove blanks
	lst_Merge_Guru = list(filter(None, lst_Merge_Guru))
	numCaps = len(lst_Merge_Guru)
	print(f"There are {numCaps} TICKERS in this page")


There are 375 links in this page
There are 48 TICKERS in this page


In [61]:
	# Remove duplicates
set_Merge_Guru =  set(lst_Merge_Guru)

#	Convert set to list to append 'END' below
lst_Merge_Guru = list(set_Merge_Guru)


print(f'There are {len(lst_Merge_Guru)} in List from Guru', lst_Merge_Guru)

There are 14 in List from Guru ['LAD', 'ETF', 'RGLD', 'FAQ', 'EAT', 'ITW', 'JHG', 'NVR', 'USA', 'BALL', 'STLA', 'API', 'GT', 'UK']


In [62]:
#   NYSE ONLY FILTER
# Append END while not END to RES_LST

tkr = '' 

def txtStripper(dcf):
    dcf = dcf.replace('\n', '')
    dcf = dcf.strip()
    print((tkr), ':',  (dcf))

lst_Merge_Guru.append('END')

#	Remove duplicates
# set_Merge =  set(lst_Merge)
# print(set_Merge)

while tkr != 'END':
    try:
        for l in lst_Merge_Guru:    
                    
                    tkr = l
                    
                    baseUrl = 'https://site.financialmodelingprep.com/discounted-cash-flow-model/{}'
                    url = baseUrl.format(str(tkr))
                    response = urllib.request.urlopen(url)
                    url_contents = response.read()

                    soup = BeautifulSoup(url_contents, "html.parser")
                    dcf = soup.find("div", {"class": "global-button2"}).text

# START CLEAN UP
                    # if 'overvalue' in dcf or  'Stock DCF: - | ' in dcf:
                    #     lst_Merge.remove(tkr)
                    if '- |' in dcf:
                       lst_Merge_Guru.remove(tkr) 
                    elif 'overvalue' in dcf:
                        lst_Merge_Guru.remove(tkr) 
                    elif dcf is None: 
                        lst_Merge_Guru.remove(tkr)
                    else:
                     txtStripper(dcf)



    except AttributeError:
        print('No' , (tkr) )
        lst_Merge_Guru.remove(tkr)
        print('Ammended tkr: ' , lst_Merge_Guru)

print('Done')
print('Appended Length: ' , len(lst_Merge_Guru))


EAT : Stock DCF: 143.35 | 34.57 | undervalue
NVR : Stock DCF: 6,226.04 | 4703.25 | undervalue
STLA : Stock DCF: 404.50 | 14.58 | undervalue
No ETF
Ammended tkr:  ['FAQ', 'EAT', 'JHG', 'NVR', 'BALL', 'STLA', 'GT', 'END']
No FAQ
Ammended tkr:  ['EAT', 'JHG', 'NVR', 'BALL', 'STLA', 'GT', 'END']
EAT : Stock DCF: 143.35 | 34.57 | undervalue
JHG : Stock DCF: 73.53 | 25.23 | undervalue
NVR : Stock DCF: 6,226.04 | 4703.25 | undervalue
EAT : Stock DCF: 143.35 | 34.57 | undervalue
JHG : Stock DCF: 73.53 | 25.23 | undervalue
NVR : Stock DCF: 6,226.04 | 4703.25 | undervalue
STLA : Stock DCF: 404.50 | 14.58 | undervalue
No END
Ammended tkr:  ['EAT', 'JHG', 'NVR', 'STLA']
Done
Appended Length:  4


In [63]:
lst_Merge_Guru

['EAT', 'JHG', 'NVR', 'STLA']

In [64]:
mergedTkrs = ''

overVal = []
underVal = []
nanList = []

dict_CurrentPrice = {}
dict_Valuation = {}
dict_Discount = {}
dict_UnderBy = {}

# tkr, $, evaluation
#lst_Comparison = []

In [65]:
# DEF   remove newline and spaces
def txtValueFilter(dcf):
    dcf = dcf.replace('\n', '')
    dcf = dcf.replace(',', '')
    dcf = dcf.strip()

    if dcf is None:
        lst_Merge_Guru.remove(mergedTkrs)

    if 'Stock DCF: - | ' in dcf: # NAN values in ticker price. instead of try: block
        nanList.append(mergedTkrs)

    # elif 'overvalue' in dcf:
    #     overVal.append(mergedTkrs) # append to list
        
        
    elif 'undervalue' in dcf:
         underVal.append(mergedTkrs) # append to list
         
         EvaluateDCF(dcf)
               
    else:
        print('No Value Statement: code Throw')

    print(mergedTkrs, ':',  (dcf))



In [66]:
# DEF   CALCULATE VALUE DIFFERENCE

def EvaluateDCF(tstStr):
    tstStr = tstStr.replace('Stock DCF:', '')
    tstStr = tstStr.replace('|', '-', 1)
    tstStr = tstStr.replace('|', '')
    tstStr = tstStr.replace('overvalue', '')
    tstStr = tstStr.replace('undervalue', '')
    
    tstStr = tstStr.split('-')

    eval = float(tstStr[0]) - float(tstStr[1])
    eval = (round(eval, 2))

    currentPrice =  float(tstStr[0])
    valuation = float(tstStr[1])
    discount = currentPrice - valuation
    underBy = round((valuation / currentPrice) * 100, 2)


    dict_CurrentPrice.update( {mergedTkrs : tstStr[0]}) 
    dict_Valuation.update({mergedTkrs : discount})
    dict_Discount.update({mergedTkrs : valuation})
    dict_UnderBy.update({mergedTkrs : underBy})

    #lst_Comparison.append(mergedTkrs, currentPrice, eval )
    
    print (mergedTkrs, 'Undervalued by: $', valuation)


In [67]:
#   CALC LOOP

for mergedTkrs in lst_Merge_Guru:
    try:
        baseUrl = 'https://site.financialmodelingprep.com/discounted-cash-flow-model/{}'
        url = baseUrl.format(str(mergedTkrs))
        response = urllib.request.urlopen(url)
        url_contents = response.read()

        soup = BeautifulSoup(url_contents, "html.parser")
        dcf = soup.find("div", {"class": "global-button2"}).text

        print('error checker: ',mergedTkrs, dcf)

        txtValueFilter(dcf) 
    
    except AttributeError:
        print('No text for ' , (mergedTkrs) )
        lst_Merge_Guru.remove(mergedTkrs)

    except KeyError: 'END'

print('CALC DONE. ')



error checker:  EAT 
                Stock DCF: 143.35 | 34.57 | undervalue
            
EAT Undervalued by: $ 34.57
EAT : Stock DCF: 143.35 | 34.57 | undervalue
error checker:  JHG 
                Stock DCF: 73.53 | 25.23 | undervalue
            
JHG Undervalued by: $ 25.23
JHG : Stock DCF: 73.53 | 25.23 | undervalue
error checker:  NVR 
                Stock DCF: 6,226.04 | 4703.25 | undervalue
            
NVR Undervalued by: $ 4703.25
NVR : Stock DCF: 6226.04 | 4703.25 | undervalue
error checker:  STLA 
                Stock DCF: 404.50 | 14.58 | undervalue
            
STLA Undervalued by: $ 14.58
STLA : Stock DCF: 404.50 | 14.58 | undervalue
CALC DONE. 


In [68]:
dict_UnderBy

{'EAT': 24.12, 'JHG': 34.31, 'NVR': 75.54, 'STLA': 3.6}

In [37]:
underVal


['NVR', 'STLA', 'JHG', 'EAT']

In [38]:

dict_CurrentPrice

{'NVR': ' 6226.04 ', 'STLA': ' 404.50 ', 'JHG': ' 73.53 ', 'EAT': ' 143.35 '}

In [39]:
# v = dict_Valuation.values()
# float([x for x in v][0])

dict_Valuation

{'NVR': 1522.79, 'STLA': 389.92, 'JHG': 48.3, 'EAT': 108.78}

In [40]:

dict_Discount

{'NVR': 4703.25, 'STLA': 14.58, 'JHG': 25.23, 'EAT': 34.57}

In [70]:
df_Value = pd.DataFrame([dict_CurrentPrice, dict_Valuation, dict_Discount, dict_UnderBy]).T
df_Value.columns = ['Price', 'Value', 'Discount', '% Under']

df_Value

,Price,Value,Discount,% Under
EAT,143.35,108.78,34.57,24.12
JHG,73.53,48.3,25.23,34.31
NVR,6226.04,1522.79,4703.25,75.54
STLA,404.50,389.92,14.58,3.6


In [26]:
df_Value = df_Value.astype(float)

In [27]:
df_Value.dtypes

Price       float64
Value       float64
Discount    float64
dtype: object

In [28]:
#   DROP EXPENCIVE STOCKS

# Get names of indexes for which column //Age has value 30
indxNames = df_Value[df_Value['Price'] > 150].index
# Delete these row indexes from dataFrame
df_Value.drop(indxNames , inplace=True)
df_Value

,Price,Value,Discount
JHG,73.53,48.30,25.23
EAT,143.35,108.78,34.57


### start from here

In [ ]:
sumV = df_Value['Value'].sum()
float(sumV)
round(sumV,2)

In [ ]:
#df_Value['Value'].round(2)

In [ ]:
89.795 in df_Value is float

In [ ]:
dfN = df_Value[df_Value['Price'] > 150]

In [ ]:
df_Value.plot(figsize=      (10,6))